In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/urldata.csv')
data.sample(10)

,Unnamed: 0,url,label,result
47254,47254,https://www.flaggedrevs.labs.wikimedia.org/wik...,benign,0
100055,100055,https://www.spineart.blogspot.com/,benign,0
256437,256437,https://www.mylife.com/tedmacnish,benign,0
93280,93280,https://www.rentalo.com/64648/sheratonmidwest....,benign,0
148751,148751,https://www.beerincanada.ca/,benign,0
190057,190057,https://www.facebook.com/people/Carmen-Daigle/...,benign,0
147264,147264,https://www.baseball-reference.com/players/j/j...,benign,0
414788,414788,http://siriwat.co.th/zie6vv,malicious,1
369256,369256,http://surfandearned.top/admi/pagedoc/page/,malicious,1
51338,51338,https://www.genforum.genealogy.com/crow/all.html,benign,0


In [ ]:
data.shape

(450176, 4)

In [ ]:
data.isnull().sum()

,0
Unnamed: 0,0
url,0
label,0
result,0


In [ ]:
data.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
data.sample(5)

,url,label,result
148147,https://www.bbc.co.uk/films/2005/06/10/the_cze...,benign,0
23471,https://www.dictionary.reference.com/browse/An...,benign,0
253557,https://www.mutigers.com/sports/m-basebl/spec-...,benign,0
58044,https://www.icehockey.wikia.com/wiki/Joe_Malone,benign,0
179583,https://www.ellenchou.com/,benign,0


In [ ]:
data.duplicated().sum()

0

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
sample_data = data.sample(100)
sample_data.head()

,url,label,result
212426,https://www.homegain.com/homes/Fort-Wayne-IN,benign,0
45007,https://www.familypedia.wikia.com/wiki/List_of...,benign,0
28979,https://www.en.wikipedia.org/wiki/1994_Canadia...,benign,0
81254,https://www.nwda-db.wsulibs.wsu.edu/findaid/ar...,benign,0
366848,http://asiandream.ru/plugins/user/ii/domain/do...,malicious,1


In [ ]:
def fetch_webpage_content(url):
    try:
        #response = requests.get(url)
        headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            return response.text
        else:
            return None
    except Exception as e:
        print(f"Error fetching webpage content: {e}")
        return None

In [ ]:
sample_data['content']=sample_data['url'].apply(lambda x: fetch_webpage_content(x))

Error fetching webpage content: HTTPSConnectionPool(host='www.familypedia.wikia.com', port=443): Max retries exceeded with url: /wiki/List_of_counties_in_Ohio (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'www.familypedia.wikia.com'. (_ssl.c:1007)")))
Error fetching webpage content: HTTPSConnectionPool(host='www.en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/1994_Canadian_Grand_Prix (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fb006b638b0>: Failed to resolve 'www.en.wikipedia.org' ([Errno -2] Name or service not known)"))
Error fetching webpage content: HTTPSConnectionPool(host='www.nwda-db.wsulibs.wsu.edu', port=443): Max retries exceeded with url: /findaid/ark:/80444/xv32838 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fb006b62c80>: Failed to resolve 'www.nwda-db.wsulibs.wsu.edu' ([Er

In [ ]:
sample_data[sample_data['content'].isnull()]

,url,label,result,content
45007,https://www.familypedia.wikia.com/wiki/List_of...,benign,0,None
28979,https://www.en.wikipedia.org/wiki/1994_Canadia...,benign,0,None
81254,https://www.nwda-db.wsulibs.wsu.edu/findaid/ar...,benign,0,None
366848,http://asiandream.ru/plugins/user/ii/domain/do...,malicious,1,None
70541,https://www.mbd.scout.com/mb.aspx?s=262&f=2079,benign,0,None
...,...,...,...,...
163345,https://www.cinemorgue2.com/dougsavant.html,benign,0,None
367959,http://ecogreentec.com.au/dozi/index.htm,malicious,1,None
131749,https://www.alternative-worlds.com/2010/11/14/...,benign,0,None
48768,https://www.fp.tcsn.net/jackie/robert_pinsky.htm,benign,0,None


In [ ]:
temp_data = sample_data

In [ ]:
temp_data.shape

(100, 4)

In [ ]:
temp_data.dropna( inplace=True)

In [ ]:
temp_data.shape

(26, 4)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
def extract_text(content):
  soup = BeautifulSoup(content, 'lxml-xml')
  text_content = soup.get_text()
  return text_content

In [ ]:
temp_data['content']=temp_data['content'].apply(lambda x: extract_text(x))

In [ ]:
X = temp_data['content']
y = temp_data['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape

((20,), (6,))

In [ ]:
X_train

,content
123934,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLa...
254222,
261065,
183037,
142729,\n\n\n\n\n.async-hide { opacity: 0 !important}...
270001,"\n\n\n\n\n\n\n\nvar whoparam=whoparam||[[""AcNo..."
141580,"\n\n45a2713f data-node=""www19.us.archive.org"">..."
387307,\n\n\n\n\n\n\n\n\n\n\n\n
320790,"\n\n\n(function (w, d, s, l, i) {\n ..."
337331,


In [ ]:
# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the training data and transform the text
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the test data (using the same vectorizer, do not fit again)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
X_test_tfidf

<6x29016 sparse matrix of type '<class 'numpy.float64'>'
	with 1416 stored elements in Compressed Sparse Row format>

In [ ]:
# Initialize the Naive Bayes classifier
clf = MultinomialNB()

# Train the classifier on the TF-IDF transformed training data
clf.fit(X_train_tfidf, y_train)


MultinomialNB()

In [ ]:
# Make predictions on the test data
y_pred = clf.predict(X_test_tfidf)

# Evaluate accuracy and other metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8333333333333334
Classification Report:
               precision    recall  f1-score   support

      benign       0.83      1.00      0.91         5
   malicious       0.00      0.00      0.00         1

    accuracy                           0.83         6
   macro avg       0.42      0.50      0.45         6
weighted avg       0.69      0.83      0.76         6



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
def predict_url(url):
    # Fetch the webpage content
    webpage_content = fetch_webpage_content(url)
    if webpage_content is None:
        return "Error fetching webpage content"
        # Extract the text content from the webpage
    text_content = extract_text(webpage_content)
    vector = vectorizer.transform(X_test)
    prediction = clf.predict(vector)
    if prediction[0] == 1:
        return "Phishing URL"
    else:
        return "Legitimate URL"


In [ ]:
print(predict_url('http://ww1.zoomato.com/'))

Legitimate URL


In [ ]:
import joblib

# Save the trained model and vectorizer
joblib.dump(clf, 'naive_bayes_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']